In [88]:
"""
    Debugging Notes: 
    Check if particle update working properly, seems to not update, only shows based on starting seed/value of particle
    >Current position values all zero out after 1st iteration, fix this
    
    ####################################################################################################
    
    Add Values on Antonio and Martin Base Values Below (Load Capacity, Max Off, Min On)
    L0: 320, 4, 2
    L1: 200, 4, 2
    L2: 80, 4, 2
    L3: 84, 4, 2
    L4: 100, 4, 2
    L5: 160, 4, 2
    L6: 100, 3, 1
    L7: 60, 3, 1
    L8: 200, 3, 1
    L9: 40, 4, 1
    
    Req Curtailment: (T0, T1, T2... T15)
    110, 230, 455, 680, 770, 800, 750, 640, 590, 610, 660, 680, 570, 410, 230, 135
    
    
    ###################################################################################################
    
    To Do List:
    Sensitivity Analysis for penalty multipliers
    
    Current Values
    Overcurtail:
    Undercurtail:
    Over Energy Limit:
    
    Sensitivity Analysis for search parameters C1 C2 W
    
    Current Values
    C1: 2
    C2: 2
    C3: 2
"""



# Import modules
import numpy as np

# Import sphere function as objective function
from pyswarms.utils.functions.single_obj import sphere as f

# Import backend modules
import pyswarms.backend as P
from pyswarms.backend.topology import Star
#import pyswarms as ps
#import pyswarms.discrete.binary as PSO

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
my_topology = Star() # The Topology Class
my_options = {'c1': 2, 'c2': 2, 'w': 2}
c1 = 2
c2 = 2
w = 2
#set dimension to 10 loads x 16 timeslots = 160
my_swarm = P.create_swarm(n_particles=10, dimensions=160, binary=True, discrete=True, options=my_options) # The Swarm Class

#Initialize cost to array of inf for max value, to_append is manually set below. Create temp storade for pbest cost as well
x = 0
temp_pbest_cost = []
to_append = np.array([float('inf')])

for x in range(my_swarm.n_particles):
    my_swarm.pbest_cost = np.append(my_swarm.pbest_cost, to_append)
    my_swarm.current_cost = np.append(my_swarm.current_cost, to_append)
    temp_pbest_cost = np.append(temp_pbest_cost, to_append)




#Printing Values for debugging
#print('The following are the attributes of our swarm: {}'.format(my_swarm.__dict__.keys()))
#print("# of particles:",my_swarm.n_particles)
#print("dimensions:",my_swarm.dimensions)
print("pos \n",my_swarm.position)
#print("velo \n",my_swarm.velocity)
#print("current cost \n",my_swarm.current_cost)
#print("pbest pos \n",my_swarm.pbest_pos)
#print("pbest cost \n",my_swarm.pbest_cost)
#print("best cost \n",my_swarm.best_cost)

pos 
 [[1 1 0 ... 0 1 1]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 0 ... 1 1 1]
 [1 0 0 ... 0 1 1]
 [1 0 0 ... 1 1 1]]


In [90]:
#Position computation, BPSO computation for position

def _compute_position(swarm):
    for i in range(swarm.n_particles):                  
        for j in range(my_swarm.dimensions):
            if _sigmoid(swarm.velocity[i][j]) < np.random.uniform(low=0.0, high=1.0):
                swarm.position[i][j] = 1
            else:
                swarm.position[i][j] = 0   
    return swarm.position

def _sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [91]:
#Velocity computation, BPSO computation for velocity

def _compute_velocity(swarm,c1,c2,w):
    for i in range(my_swarm.n_particles):                  
        for j in range(my_swarm.dimensions):
            swarm.velocity[i][j] = w*swarm.velocity[i][j] + \
            [c1*np.random.uniform(low=0.0, high=1.0)*swarm.pbest_pos[i][j]]-[c1*np.random.uniform(low=0.0, high=1.0)*swarm.position[i][j]] + \
            [c2*np.random.uniform(low=0.0, high=1.0)*my_swarm.best_cost]-[c2*np.random.uniform(low=0.0, high=1.0)*swarm.position[i][j]]
    return swarm.velocity

In [92]:
#Taken from pyswarms.backend.operators
def _compute_pbest(swarm,temp_pbest_cost):
    try:
        # Infer dimensions from positions
        dimensions = swarm.dimensions
        
        # Create a 1-D and 2-D mask based from comparisons
        # Creating 1-D mask
        mask_cost = swarm.current_cost < temp_pbest_cost
        #print("maskcost:\n",mask_cost)|
        
        # Extension of 1-D mask to 2-D mask
        mask_pos = np.repeat(mask_cost[:, np.newaxis], dimensions, axis=1)
        #print("maskpos:\n",mask_pos)
        
        # Apply masks, copy into 2nd variable if 3rd variable fits conditions
        new_pbest_pos = np.where(~mask_pos, swarm.pbest_pos, swarm.position)
        new_pbest_cost = np.where(~mask_cost, temp_pbest_cost, swarm.current_cost)
        
        #print("swarm current pos:",swarm.position)
        #print("new swarm pbest pos:",new_pbest_pos)
        #print("new swarm pbest cost:",new_pbest_cost)
        
    except AttributeError:
        rep.logger.exception(
            "Please pass a Swarm class. You passed {}".format(type(swarm))
        )
        raise
    else:
        return (new_pbest_pos, new_pbest_cost)

In [93]:
#penalty for curtailing more than necessary (lighter penalty)
def overcurtail(x):
    
    #timeslotrange = 160 elements per particle divided by 10 particles
    timeslotrange = 16
    penaltyvalue = 1000000
    totalpenalty = 0
    
    #divide array into 10 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    
    #invert array 1 and 0, count 1s to find number of 0s
    inv = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    L0 = np.subtract(inv,L0)
    L1 = np.subtract(inv,L1)
    L2 = np.subtract(inv,L2)
    L3 = np.subtract(inv,L3)
    L4 = np.subtract(inv,L4)
    L5 = np.subtract(inv,L5)
    L6 = np.subtract(inv,L6)
    L7 = np.subtract(inv,L7)
    L8 = np.subtract(inv,L8)
    L9 = np.subtract(inv,L9)
    
    #set maximum off time per load here
    limitL0 = 4
    limitL1 = 4
    limitL2 = 4
    limitL3 = 4
    limitL4 = 4
    limitL5 = 4
    limitL6 = 3
    limitL7 = 3
    limitL8 = 3
    limitL9 = 4
    
    #count for off time here, add 1 penalty value if violated
    if np.sum(L0) > limitL0:
        totalpenalty += 1
    if np.sum(L1) > limitL1:
        totalpenalty += 1
    if np.sum(L2) > limitL2:
        totalpenalty += 1
    if np.sum(L3) > limitL3:
        totalpenalty += 1
    if np.sum(L4) > limitL4:
        totalpenalty += 1
    if np.sum(L5) > limitL5:
        totalpenalty += 1
    if np.sum(L6) > limitL6:
        totalpenalty += 1
    if np.sum(L7) > limitL7:
        totalpenalty += 1
    if np.sum(L8) > limitL8:
        totalpenalty += 1
    if np.sum(L9) > limitL9:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [94]:
#penalty for undercurtailing (should be high multiplier to ensure output is not all zero)
def undercurtail(x):
    
    #timeslotrange = 160 elements per particle divided by 10 particles
    timeslotrange = 16
    penaltyvalue = 1000000
    totalpenalty = 0
    
    #divide array into 10 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    
    #set minimum on time per load here
    limitL0 = 2
    limitL1 = 2
    limitL2 = 2
    limitL3 = 2
    limitL4 = 2
    limitL5 = 2
    limitL6 = 1
    limitL7 = 1
    limitL8 = 1
    limitL9 = 1
    
    #count for ontime here, add 1 penalty value if violated
    if np.sum(L0) < limitL0:
        totalpenalty += 1
    if np.sum(L1) < limitL1:
        totalpenalty += 1
    if np.sum(L2) < limitL2:
        totalpenalty += 1
    if np.sum(L3) < limitL3:
        totalpenalty += 1
    if np.sum(L4) < limitL4:
        totalpenalty += 1
    if np.sum(L5) < limitL5:
        totalpenalty += 1
    if np.sum(L6) < limitL6:
        totalpenalty += 1
    if np.sum(L7) < limitL7:
        totalpenalty += 1
    if np.sum(L8) < limitL8:
        totalpenalty += 1
    if np.sum(L9) < limitL9:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [95]:
#penalty for exceeding energy limit per timeslot
def overenergylimit(x):
    penaltyvalue = 1000000
    totalpenalty = 0
    
    #set limit for each TS in kW here
    TS0limit = 110
    TS1limit = 220
    TS2limit = 455
    TS3limit = 680
    TS4limit = 770
    TS5limit = 800
    TS6limit = 750
    TS7limit = 640
    TS8limit = 590
    TS9limit = 610
    TS10limit = 660
    TS11limit = 680
    TS12limit = 570
    TS13limit = 410
    TS14limit = 230
    TS15limit = 135
    
    #set consumption for each load in kW here
    cons0 = 320
    cons1 = 200
    cons2 = 80
    cons3 = 84
    cons4 = 100
    cons5 = 160
    cons6 = 100
    cons7 = 60
    cons8 = 200
    cons9 = 40
    
    #set self generated energy for each load in kW here
    self0 = 0
    self1 = 0
    self2 = 0
    self3 = 0
    self4 = 0
    self5 = 0
    self6 = 0
    self7 = 0
    self8 = 0
    self9 = 0
    
    consumption = [cons0-self0, cons1-self1, cons2-self2, cons3-self3, cons4-self4, cons5-self5, cons6-self6, cons7-self7, cons8-self8, cons9-self9]
    
    #cut particles into timeslots here, from L0 to L15. Note: 160 = 10 loads x 16 timeslots
    TS0 = x[0:160:16]
    TS1 = x[1:160:16]
    TS2 = x[2:160:16]
    TS3 = x[3:160:16]
    TS4 = x[4:160:16]
    TS5 = x[5:160:16]
    TS6 = x[6:160:16]
    TS7 = x[7:160:16]
    TS8 = x[8:160:16]
    TS9 = x[9:160:16]
    TS10 = x[10:160:16]
    TS11 = x[11:160:16]
    TS12 = x[12:160:16]
    TS13 = x[13:160:16]
    TS14 = x[14:160:16]
    TS15 = x[15:160:16]
    
    #check for instances of overconsumption
    if np.sum(np.multiply(TS0,consumption)) > TS0limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS1,consumption)) > TS1limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS2,consumption)) > TS2limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS3,consumption)) > TS3limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS4,consumption)) > TS4limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS5,consumption)) > TS5limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS6,consumption)) > TS6limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS7,consumption)) > TS7limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS8,consumption)) > TS8limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS9,consumption)) > TS9limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS10,consumption)) > TS10limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS11,consumption)) > TS11limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS12,consumption)) > TS12limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS13,consumption)) > TS13limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS14,consumption)) > TS14limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS15,consumption)) > TS15limit:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [96]:
#test function, 2D version, to be implemented per element (Element is 1D array)
def obj_func(x):
    
    #timeslotrange = 160 elements per particle divided by 10 particles
    timeslotrange = 16
    
    #set costs per kW here
    DUcost = 10
    SGEcost = 5
    
    #set consumption for each load in kW here
    cons0 = 320
    cons1 = 200
    cons2 = 80
    cons3 = 84
    cons4 = 100
    cons5 = 160
    cons6 = 100
    cons7 = 60
    cons8 = 200
    cons9 = 40
    
    #set self generated energy for each load in kW here
    self0 = 0
    self1 = 0
    self2 = 0
    self3 = 0
    self4 = 0
    self5 = 0
    self6 = 0
    self7 = 0
    self8 = 0
    self9 = 0

    #divide array into 10 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    
    f =  np.sum(np.array(L0)*DUcost*(cons0-self0)) + np.sum(np.array(L0)*SGEcost*(self0)) +\
    np.sum(np.array(L1)*DUcost*(cons1-self1)) + np.sum(np.array(L1)*SGEcost*(self1)) +\
    np.sum(np.array(L2)*DUcost*(cons2-self2)) + np.sum(np.array(L2)*SGEcost*(self2)) +\
    np.sum(np.array(L3)*DUcost*(cons3-self3)) + np.sum(np.array(L3)*SGEcost*(self3)) +\
    np.sum(np.array(L4)*DUcost*(cons4-self4)) + np.sum(np.array(L4)*SGEcost*(self4)) +\
    np.sum(np.array(L5)*DUcost*(cons5-self5)) + np.sum(np.array(L5)*SGEcost*(self5)) +\
    np.sum(np.array(L6)*DUcost*(cons6-self6)) + np.sum(np.array(L6)*SGEcost*(self6)) +\
    np.sum(np.array(L7)*DUcost*(cons7-self7)) + np.sum(np.array(L7)*SGEcost*(self7)) +\
    np.sum(np.array(L8)*DUcost*(cons8-self8)) + np.sum(np.array(L8)*SGEcost*(self8)) +\
    np.sum(np.array(L9)*DUcost*(cons9-self9)) + np.sum(np.array(L9)*SGEcost*(self9)) +\
    overcurtail(x) + undercurtail(x) + overenergylimit(x)
    
    return f


In [97]:
#print("Initial Value current cost:\n",my_swarm.current_cost)
#print("Initial Value current pos:\n",my_swarm.position)
#print("Initial Value pbest pos:\n",my_swarm.pbest_pos)
#print("Initial Value pbest cost:\n",my_swarm.pbest_cost)

iterations = 100 # Set 100 iterations
for i in range(iterations):
    
    # Part 0: Compute current cost
    j=0
    for j in range(my_swarm.n_particles):
        my_swarm.current_cost[j] = obj_func(my_swarm.position[j][:]) # Compute current cost (using obj func)
        temp_pbest_cost[j] = obj_func(my_swarm.pbest_pos[j][:])  # Compute personal best cost (using obj func)
    
    #my_swarm.pbest_cost = f(my_swarm.pbest_pos) # Compute current cost (using built in sphere func)
    #print("-------------------------------")
    #print("Pbest pos:\n",my_swarm.pbest_pos)
    #print("Pbest cost:\n",my_swarm.pbest_cost)
    print("Current pos:\n",my_swarm.position)
    #print("Current cost:\n", my_swarm.current_cost)
   
    #Update personal best pos/cost 
    my_swarm.pbest_pos, my_swarm.pbest_cost = _compute_pbest(my_swarm,temp_pbest_cost) # Update and store
    
    # Part 2: Update global best
    # Note that gbest computation is dependent on your topology
    if np.min(my_swarm.pbest_cost) < my_swarm.best_cost:
        my_swarm.best_pos, my_swarm.best_cost = my_topology.compute_gbest(my_swarm)

    # Let's print our output
    if i%20==0:
        print('Iteration: {} | my_swarm.best_cost: {:.4f}'.format(i+1, my_swarm.best_cost))
        print("Pbest cost:\n",my_swarm.pbest_cost)
        
    # Part 3: Update position and velocity matrices
    # Note that position and velocity updates are dependent on your topology
    
    #Fix the velocity and position computation for Binary application
    my_swarm.velocity = _compute_velocity(my_swarm,c1,c2,w)
    my_swarm.position = _compute_position(my_swarm) 
    
 
print('The best cost found by our swarm is: {:.4f}'.format(my_swarm.best_cost))
#print('The best position found by our swarm is: {}'.format(my_swarm.best_pos))

#divide array into 10 loads x 16 timeslots
timeslotrange = 16
L0 = my_swarm.best_pos[0:timeslotrange]
L1 = my_swarm.best_pos[timeslotrange:(timeslotrange*2)]
L2 = my_swarm.best_pos[(timeslotrange*2):(timeslotrange*3)]
L3 = my_swarm.best_pos[(timeslotrange*3):(timeslotrange*4)]
L4 = my_swarm.best_pos[(timeslotrange*4):(timeslotrange*5)]
L5 = my_swarm.best_pos[(timeslotrange*5):(timeslotrange*6)]
L6 = my_swarm.best_pos[(timeslotrange*6):(timeslotrange*7)]
L7 = my_swarm.best_pos[(timeslotrange*7):(timeslotrange*8)]
L8 = my_swarm.best_pos[(timeslotrange*8):(timeslotrange*9)]
L9 = my_swarm.best_pos[(timeslotrange*9):(timeslotrange*10)]

print('Schedule of Load 0:', L0)
print('Schedule of Load 1:', L1)
print('Schedule of Load 2:', L2)
print('Schedule of Load 3:', L3)
print('Schedule of Load 4:', L4)
print('Schedule of Load 5:', L5)
print('Schedule of Load 6:', L6)
print('Schedule of Load 7:', L7)
print('Schedule of Load 8:', L8)
print('Schedule of Load 9:', L9)



Current pos:
 [[1 1 0 ... 0 1 1]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 0 ... 1 1 1]
 [1 0 0 ... 0 1 1]
 [1 0 0 ... 1 1 1]]
Iteration: 1 | my_swarm.best_cost: 18091280.0000
Pbest cost:
 [21114880. 21091840. 18091280. 23119760. 19107680. 21111040. 20094200.
 23111480. 20101920. 22115160.]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0

Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Iteration: 61 | my_swarm.best_cost: 18091280.0000
Pbest cost:
 [20000000. 20000000. 18091280. 20000000. 19107680. 20000000. 20000000.
 20000000. 20000000. 20000000.]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Current pos:
 [[0 0 0 ... 0 0 